In [16]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

#minta szerint
#szerintem ez a jó
def timeseries_second():
    last_keydown = ''
    timeseries = []
    row = data_in.readline()
    row = row.split()
    while row:
        if row[0] == 'keydown':
            
            if last_keydown :
                timeseries.append(int(row[5])-int(last_keydown))
            last_keydown = row[5]
        row = data_in.readline()
        row = row.split()
    return timeseries
    
data_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-raw-data.txt","r")
training_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-train-labels.txt","r")
test_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-test-hypothetic-labels.txt","r")
data_x = []
data_y = []
row = data_in.readline().strip()
while row:
    if 'TYPING PATTERN' in row:
        data_x.append(timeseries_second())
    row = data_in.readline().strip()
    
data_in.close()
#separating the train and test dataset
train_x = []
train_y = []
test_x = []
test_y = []
test_index = []
row = training_in.readline().strip()
while row:
    row = row.split(',')
    train_x.append(data_x[int(row[0])-1])
    train_y.append(int(row[1]))
    row = training_in.readline()
training_in.close()
#the test datasets
row = test_in.readline().strip()
while row:
    row = row.split(',')
    test_x.append(data_x[int(row[0])-1])
    test_index.append(int(row[0]))
    test_y.append(int(row[1]))
    row = test_in.readline()
test_in.close()

data_length = []
for data in data_x:
    data_length.append(len(data))
    
distance, path = fastdtw(train_x[0], train_x[25], dist=euclidean)
print(distance)

40455.0


In [17]:
import itertools
def most_common(L):
  groups = itertools.groupby(sorted(L))
  def _auxfun((item, iterable)):
    return len(list(iterable)), -L.index(item)
  return max(groups, key=_auxfun)[0]

In [18]:
def timeseries_first():
    keydown = ''
    timeseries = []
    row = data_in.readline()
    row = row.split()
    while row:
        if row[0] == 'keydown':
            keydown = row[5]
        if keydown and row[0] == 'keyup':
            timeseries.append(int(row[5])-int(keydown))
            keydown = ''
        row = data_in.readline()
        row = row.split()
    return timeseries
    
data_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-raw-data.txt","r")
training_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-train-labels.txt","r")
test_in = open("C:/Users/valikund/Desktop/Challange/keystrokes-12users-test-hypothetic-labels.txt","r")
data_x_1 = []
data_y_1 = []
row = data_in.readline().strip()
while row:
    if 'TYPING PATTERN' in row:
        data_x_1.append(timeseries_first())
    row = data_in.readline().strip()
    
data_in.close()
#separating the train and test dataset
train_x_1 = []
train_y_1 = []
test_x_1 = []
test_y_1 = []
row = training_in.readline().strip()
while row:
    row = row.split(',')
    train_x_1.append(data_x[int(row[0])-1])
    train_y_1.append(int(row[1]))
    row = training_in.readline()
training_in.close()
#the test datasets
row = test_in.readline().strip()
while row:
    row = row.split(',')
    test_x_1.append(data_x[int(row[0])-1])
    test_y_1.append(int(row[1]))
    row = test_in.readline()
test_in.close()

data_length = []
for data in data_x:
    data_length.append(len(data))
    
distance, path = fastdtw(train_x[0], train_x[25], dist=euclidean)
print(distance)

40455.0


In [4]:
def predict(train_x,train_y,test_x, k):
    index = 0
    knn = []
    knn_label = []
    for x in train_x:
        distance, path = fastdtw(x, test_x, dist=euclidean)
        if len(knn)<k :
            knn.append(distance)
            knn_label.append(train_y[index])
        elif max(knn)>distance:
            remove_item = knn.index(max(knn))
            knn.pop(remove_item)
            knn_label.pop(remove_item)
            knn.append(distance)
            knn_label.append(train_y[index])            
        index = index + 1
    return knn_label

In [20]:
train_x_sorted = []
train_y_arranged = np.array(train_y)
sort_index = np.argsort(train_y_arranged)
for position in sort_index:
    train_x_sorted.append(train_x[position])
train_y.sort()   
print(train_y_arranged, train_y)

(array([ 2,  2,  2,  2,  1,  3,  2,  3,  3,  3,  3,  1,  1,  4,  4,  4,  1,
        1,  4,  4, 12, 12, 12, 12, 12, 10, 10, 10, 10, 10,  7,  7,  7,  7,
        7, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  8,  8,  8,  8,  8,  6,
        6,  6,  6,  6,  9,  9,  9,  9,  9]), [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12])


In [19]:
import numpy as np
#np.set_printoptions(threshold='nan')
dtw_array = np.empty([60,60])
for index_data,train_data in enumerate(train_x_sorted):
    for index_other,train_other in enumerate(train_x_sorted):
        distance, path = fastdtw(train_data, train_other, dist=euclidean)
        dtw_array[index_other,index_data] = distance
print(dtw_array)


[[     0.  57382.  52965. ...,  69525.  63772.  69348.]
 [ 57382.      0.  49334. ...,  47618.  43745.  57126.]
 [ 52965.  49334.      0. ...,  66888.  62667.  64530.]
 ..., 
 [ 69525.  47618.  66888. ...,      0.  29882.  37381.]
 [ 63772.  43745.  62667. ...,  29882.      0.  30760.]
 [ 69348.  57126.  64530. ...,  37381.  30760.      0.]]


In [8]:
dtw_train_average = np.ones([12,12])
for row in range(0,12):
    for column in range(0,12):
        #print(dtw_array[row*5:(row+1)*5,column*5:(column+1)*5].sum()/20)
        if row == column:
            dtw_train_average[row,column] = dtw_array[row*5:(row+1)*5,column*5:(column+1)*5].sum() /20
        else:
            dtw_train_average[row,column] = dtw_array[row*5:(row+1)*5,column*5:(column+1)*5].sum() / 25
#print(dtw_train_average)
dtw_array_refined = dtw_array
for i in range(0,12):
    for x in range(0,5):
        dtw_array_refined[i+x,i+x] = dtw_train_average[i,i]
print(dtw_array_refined)

[[ 59137.5  57382.   52965.  ...,  69525.   63772.   69348. ]
 [ 57382.   38724.1  49334.  ...,  47618.   43745.   57126. ]
 [ 52965.   49334.   50086.8 ...,  66888.   62667.   64530. ]
 ..., 
 [ 69525.   47618.   66888.  ...,      0.   29882.   37381. ]
 [ 63772.   43745.   62667.  ...,  29882.       0.   30760. ]
 [ 69348.   57126.   64530.  ...,  37381.   30760.       0. ]]


In [9]:
dtw_test = np.empty([488,60])
for index_data,train_data in enumerate(test_x):
    for index_other,train_other in enumerate(train_x_sorted):
        distance, path = fastdtw(train_data, train_other, dist=euclidean)
        dtw_test[index_data,index_other] = distance
    

In [23]:
print(test_x)

[[103L, 385L, 73L, 142L, 6L, 499L, 216L, 39L, 208L, 57L, 432L, 168L, 31L, 120L, 232L, 345L, 89L, 104L, 136L, 50L, 292L, 313L, 679L, 82L, 86L, 624L, 57L, 55L, 96L, 136L, 59L, 263L, 73L, 61L, 465L, 47L, 169L, 175L, 41L, 151L, 385L, 119L, 96L, 88L, 113L, 488L, 577L, 46L, 176L, 528L, 49L, 535L, 57L, 81L, 206L, 273L, 73L, 63L, 463L, 129L, 167L, 49L, 423L, 224L, 537L, 71L, 184L, 145L, 63L, 673L, 90L, 126L, 175L, 49L, 103L, 584L, 41L, 56L, 168L, 80L, 415L, 57L, 183L, 74L, 614L, 384L, 25L, 81L, 494L, 161L, 65L, 807L, 599L, 25L, 104L, 729L, 118L, 208L, 176L, 136L, 209L, 49L, 367L, 168L, 207L, 81L, 503L, 360L, 26L, 2L, 444L, 128L, 384L, 185L, 49L, 86L, 1536L, 49L, 159L, 256L, 74L, 542L, 137L, 199L, 73L, 119L, 49L, 455L, 144L, 65L, 103L, 1016L, 649L, 119L, 176L, 75L, 101L, 57L, 848L, 111L, 96L, 49L, 199L, 128L, 704L, 232L, 75L, 173L, 82L, 143L, 48L, 119L, 104L, 81L, 89L, 342L, 609L, 543L, 40L, 96L, 177L, 143L, 81L, 632L, 352L, 215L, 57L, 128L, 111L, 82L, 174L, 58L, 87L, 440L, 103L, 624L, 169L, 22

In [10]:
dtw_test_averaged = np.ones([488,12])
for index, test_case in enumerate(dtw_test):
    for classic in range(0,12):
        dtw_test_averaged[index,classic] = dtw_test[index,classic*5 : (classic+1)*5].sum() / 5

In [12]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
class_label = []
for i in range(1,13):
    class_label.extend([i,i,i,i,i])
clf = LogisticRegressionCV(Cs = 30)
clf = clf.fit(dtw_array_refined, class_label)
pred = clf.predict(dtw_test)
accuracy_score(test_y,pred)

0.23975409836065573

In [68]:
from sklearn.metrics import accuracy_score
pred = []
for test in dtw_test:
    dtw_series = []
    for train in dtw_array_refined:
        distance, path = fastdtw(train, test, dist=euclidean)
        dtw_series.append(distance)
    pred.append(int(dtw_series.index(min(dtw_series))/5)+1)
accuracy_score(test_y,pred)

0.18237704918032788

In [ ]:
print(dtw_test_averaged)

In [ ]:
test_index

In [113]:
out = open("submission.txt","w")

for index,y in enumerate(test_y):
    result = 6
    if y == pred[index]:
        result = 1
    else:
        result = 0
    out.write(str(test_index[index])+",")
    out.write(str(result))
    out.write("\n")
out.close()

In [15]:
np.save("dtw_array_refined",dtw_test)